## Perguntas: 
1-número de fatalidades tem diminuido com os anos? *procure pelas colunas de fataliade

2-durante a noite há mais ocorrências (18:00 a 05:00)?

3-algumas regiões tem mais ocorrências que outras?

4-Verificar em quantas cidades já aconteceram ocorrências?

5-alguns operadores têm mais ocorrências que outros?

In [ ]:
pip install pyspark

In [ ]:
pip install "dask[complete]"

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, hour
import pandas as pd
# spark = SparkSession.builder.appName("MeuApp").getOrCreate()
spark = SparkSession.builder.appName("Exemplo").config("spark.driver.memory", "4g").config("spark.executor.memory", "4g").getOrCreate()


In [ ]:
#importa dado mais rápido ao frame, pelo teste equivale ao fread do R
import dask.dataframe as dd

In [ ]:
df_ocorrencias = spark.read.option("delimiter", ";").option("header", True).csv("ocorrencia.csv")
df_ocorrencias.show(truncate=False)

In [ ]:
df_municipios = spark.read.option("delimiter", ",").option("header", True).csv("municipios.csv")
df_municipios.show(truncate=False)

In [ ]:
df_estados = spark.read.option("delimiter",",").option("header",True).csv("estados.csv")
df_estados.show(truncate=False, n=50)

Limpeza dos dados


In [ ]:
def limpar_dataframe(df):
    for col in df.columns:
        if df[col].dtype == object:  # Se a coluna for do tipo object (texto)
            df[col] = df[col] .astype(str) # converte tudo para string
            df[col] = df[col].str.upper()  # Converte para maiúsculas
            df[col] = df[col] .str.strip() # remove espaço em branco
            df[col] = df[col] .str.replace("\t", "") # se tiver o caracter \t remove
            df[col] = df[col] .apply(unidecode.unidecode)
            df[col] = df[col].str.replace('[^\w\s]', '', regex=True)  # Remove caracteres especiais

            #df[col] = df[col].apply(lambda x: unidecode.unidecode(x) if pd.notnull(x) else "")  # Remove acentuações
    return df

In [ ]:
df_estados_pandas = df_estados.toPandas();
df_ocorrencias_pandas = df_ocorrencias.toPandas();
df_municipios_pandas = df_municipios.toPandas();

In [ ]:
dfEstadoMunicipio = dd.merge(df_estados_pandas, df_municipios_pandas, on='codigo_uf')

print(f"Estado tem {len(df_estados_pandas)} linhas")
print(f"Municipio tem {len(df_municipios_pandas)} linhas")
print(f"Tem que ter a mesma quantidade de municipio o merge {len(dfEstadoMunicipio)} linhas")

In [ ]:
dfEstadoMunicipio = dfEstadoMunicipio.rename(
    columns={'codigo_uf': 'CODIGO_UF',
             'uf': 'UF',
             'nome_e': 'NOME_ESTADO',
             'latitude_e': 'LATITUDE_ESTADO',
             'longitude_e': 'LONGITUDE_ESTADO',
             'regiao': 'REGIAO',
             'codigo_ibge': 'CODIGO_IBGE',
             'nome_m': 'NOME_CIDADE',
             'latitude_m': 'LATITUDE_CIDADE',
             'longitude_m': 'LONGITUDE_CIDADE',
             'capital': 'CAPITAL',
             'siafi_id': 'SIAFI_ID',
             'ddd': 'DDD',
             'fuso_horario': 'FUSO_HORARIO'
             }
    )

In [ ]:
dfEstadoMunicipio = limpar_dataframe(dfEstadoMunicipio)

In [ ]:

dfEstadoMunicipio_pandas = dfEstadoMunicipio

municipios_nao_presentes = df_ocorrencias_pandas[~df_ocorrencias_pandas['Municipio'].isin(dfEstadoMunicipio_pandas['NOME_CIDADE'])]

print('Municípios em dfOcorrencia que não estão em dfEstadoMunicipio:')
print(municipios_nao_presentes['Municipio'].unique())

In [ ]:
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('RIO DE JANERIO', 'RIO DE JANEIRO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('RIO DE  JANEIRO', 'RIO DE JANEIRO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MACAUBA', 'MACAUBAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BOA VISTA DOS RAMOS', 'BOA VISTA DO RAMOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA SANTO INACIO  RS', 'ROSARIO DO SUL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BASE AEREA DE SANTOS', 'GUARUJA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PRAIA DE ENSEADA  SAO FRANCISCO DO SUL', 'SAO FRANCISCO DO SUL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ITABAPOANA', 'SAO FRANCISCO DE ITABAPOANA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTANA DO PARNAIBA', 'SANTANA DE PARNAIBA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ESPINGARDA', 'NUM DAAAA')


In [ ]:
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('Erradas','Corrigidas')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('NAN','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BOA VISTA DOS RAMOS','BOA VISTA DOS RAMOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PRAIA DE ENSEADA  SAO FRANCISCO DO SUL','SAO FRANCISCO DO SUL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTANA DO PARNAIBA','SANTANA DE PARNAIBA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FORA DO BRASIL','FORA DO BRASIL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CUARI','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPPINAS','CAMPINAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA FORTALEZA DE SANTA TEREZINHA','JEQUITIAI')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ITAPORANGA D AJUDA','ITAPORANGA D AJUDA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA NOVA ESPERANCA BALSAS','BALSAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SAO JOAO DEL REY','SAO JOAO DEL REI')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ESPINGARDA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BRASILIADF','BRASILIA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTA CRUZ DAS PALMAS','SANTA CRUZ DAS PALMEIRAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTA TEREZINHA DE JESUS','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('VILA BELA DA SANTISSIMA','VILA BELA DA SANTISSIMA TRINDADE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BRAISLANDIA','BRASILANDIA ')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTA VITORIA DO PAMAR','SANTA VITORIA DO PALMAR')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PARAUABEBAS','PARAUAPEBAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('HANGAR LIDER AVIACAO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ASSIS CHATEAUBRIAN','ASSIS CHATEAUBRIAND')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('LUIS EDUARDO MAGAL','LUIS EDUARDO MAGALHAES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('TRAJANO DE MORAIS','TRAJANO DE MORAES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('AEROPORTO DE GUARULHOS','GUARULHOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMOINAS','CAMPINAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('AEROPORTO DE SAO JOSE DOS CAMPOS','SAO JOSE DOS CAMPOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTA TEREZINHA DO ITAIPU','SANTA TEREZINHA DE ITAIPU')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ESTADOS UNIDOS','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('AEROPORTO DE KIMBERLEY','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA BOA VISTA QUIRINOPOLIS','QUIRINOPOLIS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BUENOS AIRES ARGENTINA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BAIRRO MULTIRAO I','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('KUALA LUMPUR MALASIA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('232548S','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SITIO SAO JOSE ZONA RURAL DE GLORIA DOESTE','GLORIA D OESTE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA FLORESTA BRASNORTE','BRASNORTE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PALIMIU','ALTO ALEGRE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPINA GRANDE ','CAMPINA GRANDE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('RIO DE  JANEIRO','RIO DE JANEIRO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('RIO DE JANERIO','RIO DE JANEIRO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA SANTO INACIO  RS','SAO BORJA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ITABAPOANA','Barra do Itabapoana')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTO ANTONIO DE LEVERGER','SANTO ANTONIO DE LEVERGER')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('COUTO DE MAGALHAES','COUTO DE MAGALHAES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTAEM','SANTAREM')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA SAO FRANCISCO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPO DE MARTE','SÃO PAULO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PONTES DE LACERDA','PONTES E LACERDA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA NOVA AURORA CAROLINA','CAROLINA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SERRRANOPOLIS','SERRANOPOLIS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('AREA RURAL','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPOS DOS GOITACAZES','CAMPOS DOS GOYTACAZES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CHAPADA DOS GUIMA','CHAPADA DOS GUIMARAES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SAO JOSE DO PINHAL','SANTO ANTONIO DO PINHAL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('RECUFE','RECIFE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('JERICOACOARA','JERICOACOARA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PARATI','PARATI')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SAO JOSE DOS CAMOPOS','SAO JOSE DOS CAMPOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BELO HORIZONE','BELO HORIZONTE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BLUMENAL','BLUMENAL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MIGRANTINOPOLIS','MIGRANTINOPOLIS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MOGIMIRIM','MOGI MIRIM')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA TRES FIGUEIRAS','IMBE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ERICO  AMAJARI','AMAJARI')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPO GRNADE','CAMPO GRANDE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPOS','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CANCUN MEXICO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('GUANAJUATO MEXICO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAIXIAS DO SUL','CAXIAS DO SUL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CIDADE DO MEXICO MEXICO','NUL')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PORTO ALEGRE ','PORTO ALEGRE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CIDADE DE UNA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CHITOSE JAPAO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SILAO GUANAJUATO MEXICO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('LISBOA PORTUGAL','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA SANTA BARBARA BOM JESUS DE GOIAS','BOM JESUS DE GOIAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FOZ DO IGUAC','FOZ DO IGUACU')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CARLANDIA','CARLANDIA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MACAUBA','MACAUBA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BASE AEREA DE SANTOS','SANTOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PEDRO JUAN CABALLERO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('NOVA ARIPUANA','NOVA ARIPUANA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PORTO ESPERIDAO','PORTO ESPERIDIAO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('NAO IDENTIFICADA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('LONDRINA PR','LONDRINA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('GAUIRA','GUAIRA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA VIGOLO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPO DOS GOYTACAZES','CAMPOS DOS GOYTACAZES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MIAMI','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('LISBOAPOTUGAL','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAPE TOWN','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SLAVADOR','SALVADOR')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MINAS GERAIS','BELO HORIZONTE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CURITBA','CURITIBA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FLOARIANOPOLIS','FLORIANOPOLIS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPOS DOS GOYTACAS','CAMPOS DOS GOYTACAZES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CONSELHEIRO LAFAYETE','CONSELHEIRO LAFAIETE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPO MORAO','CAMPO MOURAO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA INHUMAS DO CHAPADAO MG','UBERABA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPOS DOS GOYTACASES','CAMPOS DOS GOYTACAZES')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MADRID','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FAZENDA UBIRATA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CURITUBA','CURITIBA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('AGUASCALIENTES MEXICO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MONTEVIDEO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('1254','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PORO VELHO','PORTO VELHO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BIRITIBA MIRIM','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('LONGITUDE HISTORICO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MAR DEL PLATA ARGENTINA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ARGENTINA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CURUTIBA','CURITIBA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SENEGAL AFRICA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SAO JOSE DOS PINHARES','SAO JOSE DOS PINHAIS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('GUARULHOSS','GUARULHOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('JACAREPAGUA','JACAREPAGUA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MISSIONESARG','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('TUPANCI','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ALIANCA DOS TOCANTINS','ALIANCA DO TOCANTINS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAPAO DO CANOA','CAPAO DA CANOA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('YARA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('TANGARA DAQ SERRA','TANGARA DA SERRA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('GUARILHOS','GUARULHOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CASIMIRO E ABREU','CASIMIRO DE ABREU')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PARATININGA','PIRATININGA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('GAURULHOS','GUARULHOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('GRAMADINHO','RIO VERDE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('URBERLANDIA','UBERLANDIA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SOMALIA AFRICA','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MORELOS MEXICO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PARIS','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MANUEL URBANO','MANOEL URBANO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('DESCONHECIDO','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SA PAULO','SAO PAULO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('AGUAS CLARAS','BRASILIA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CAMPO GRANDEMS','CAMPO GRANDE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('EXTERIOR','NA')


In [ ]:
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MACAUBA','MACAUBAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SANTO ANTONIO DE LEVERGER','SANTO ANTONIO DO LEVERGER')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('NOVO ARIPUANA','NOVO ARIPUANA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ITAPORANGA D AJUDA','ITAPORANGA D AJUDA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('MIGRANTINOPOLIS','NOVO HORIZONTE DO OESTE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BOA VISTA DOS RAMOS','BOA VISTA DOS RAMOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('FORA DO BRASIL','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('JACAREPAGUA','RIO DE JANEIRO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BRASILANDIA ','SAO PAULO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('NUL','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('Barra do Itabapoana','SAO FRANCISCO DE ITABAPOANA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('COUTO DE MAGALHAES','COUTO DE MAGALHAES DE MINAS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('JEQUITIAI','JEQUITIBA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('JERICOACOARA','JIJOCA DE JERICOACOARA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('GLORIA D OESTE','GLORIA DOESTE')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('SÃO PAULO','SAO PAULO')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('PARATI','PARATY')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('224836S','NA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BLUMENAL','BLUMENAU')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('BOA VISTA DOS RAMOS','BOA VISTA DO RAMOS')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('NOVA ARIPUANA','NOVO ARIPUANA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('ITAPORANGA D AJUDA','ITAPORANGA DAJUDA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('CARLANDIA','INDIARA')
df_ocorrencias_pandas['Municipio'] = df_ocorrencias_pandas['Municipio'].replace('NUM DAAAA','NA')


In [ ]:
# Convertendo os DataFrames pandas para DataFrames PySpark
df_estados = spark.createDataFrame(df_estados_pandas)
df_ocorrencias= spark.createDataFrame(df_ocorrencias_pandas)
df_municipios = spark.createDataFrame(df_municipios_pandas)


### 1-número de fatalidades tem diminuido com os anos? *procure pelas colunas de fataliade


In [ ]:

# Converta a coluna Data_da_Ocorrencia para o tipo Date
df_ocorrencias = df_ocorrencias.withColumn("Data_da_Ocorrencia", F.to_date(df_ocorrencias["Data_da_Ocorrencia"], "dd/MM/yyyy"))

# Extraia o ano da coluna Data_da_Ocorrencia
df_ocorrencias = df_ocorrencias.withColumn("Ano", F.year("Data_da_Ocorrencia"))

# Calcule o número total de fatalidades por ano
fatalidades_por_ano = df_ocorrencias.groupBy("Ano").agg(
    (F.sum("Lesoes_Fatais_Tripulantes") + F.sum("Lesoes_Fatais_Passageiros") + F.sum("Lesoes_Fatais_Terceiros")).alias("Total_Fatalidades")
)

# Ordene os resultados por ano
fatalidades_por_ano = fatalidades_por_ano.orderBy(F.desc("Ano"))

# Mostre os resultados
fatalidades_por_ano.show(n=50)



### 2 - Durante a noite há mais ocorrências (18:00 a 05:00)?

In [ ]:
# Definir uma função que verifica o período do dia
def periodo_dia(hora_col):
    return F.when(
        hora_col.isNull(), F.lit(None)
    ).when(
        (hora_col >= 18) | (hora_col < 5), F.lit('noite')
    ).otherwise(F.lit('dia'))

In [ ]:
# Adicionar a nova coluna 'Periodo_do_Dia' baseada na coluna 'Hora_da_Ocorrencia'
df_ocorrencias = df_ocorrencias.withColumn('Periodo_do_Dia', periodo_dia(F.hour('Hora_da_Ocorrencia')))

# Mostrar o DataFrame resultante
#df_ocorrencias = df_ocorrencias.select("Numero_da_Ocorrencia","Classificacao_da_Ocorrencia","Hora_da_Ocorrencia","Periodo_do_Dia")
df_ocorrencias.show(truncate=False)

In [ ]:
df_periodoDia = df_ocorrencias.groupBy("Periodo_do_Dia").agg(F.count(F.col("Periodo_do_Dia")).alias("Total"))
df_periodoDia.filter((F.col("Periodo_do_Dia") != "NULL")).show(truncate=False)
#df_periodoDia.show(truncate=False)

### 3 - Algumas regiões tem mais ocorrências que outras?


In [ ]:

# Realizar o join entre df_ocorrencias e df_regioes baseado na coluna "Regiao"
df_joined_regioes = df_ocorrencias.join(df_estados, "Regiao", "inner")

# Agora, realizar a agregação no DataFrame resultante do join
df_regioes =  df_joined_regioes.groupBy("REGIAO").agg(F.count(F.col("REGIAO")).alias("Qtd_ocorrencias")).orderBy(F.desc("Qtd_ocorrencias"))

# Exibir o resultado
df_regioes.show()


### 4 - Verificar em quantas cidades já aconteceram ocorrências?

In [ ]:

# Realizar a junção dos DataFrames
# Utilizamos o método join para combinar os dois DataFrames com base em uma chave comum.
# Aqui, estamos juntando as tabelas onde a coluna 'Municipio' no DataFrame df_ocorrencias 
# corresponde à coluna 'nome' no DataFrame df_municipios.
# O parâmetro how='inner' especifica que queremos uma junção interna, 
# ou seja, apenas as linhas com correspondência em ambas as tabelas serão mantidas.
merged_df = df_ocorrencias.join(df_municipios, df_ocorrencias.Municipio == df_municipios.nome_m, how='inner')

# Contar o número de municípios distintos onde ocorreram ocorrências
# Usamos select para selecionar a coluna 'Municipio' do DataFrame resultante da junção.
# O método distinct() remove duplicatas, de modo que cada município apareça apenas uma vez.
# Finalmente, count() conta o número de linhas distintas, o que nos dá o número de municípios únicos.
num_cidades_com_ocorrencias = merged_df.select("Municipio").distinct().count()

# Imprimimos o número de cidades onde ocorreram as ocorrências.
print(f"Ocorrências já aconteceram em {num_cidades_com_ocorrencias} cidades.")

### 5 - alguns operadores têm mais ocorrências que outros?

In [ ]:
df_operadores = df_ocorrencias.groupBy("Operador_Padronizado").agg(F.count(F.col("Operador_Padronizado")).alias("Qtd")).orderBy(F.desc("Qtd"))

df_operadores.show(truncate=False, n=500)